# libraries & global vars

In [16]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

# io job_str

In [17]:
path_job_str = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 4 - career/4.3.1 jobs/"

In [18]:
filein_job_str = "job_str_1_mergeold.xlsx"

In [19]:
fileout_job_str = "job_str_2_substr"

In [20]:
job_str = pd.read_excel(path_job_str+filein_job_str,dtype="str")

In [21]:
job_str.shape

(7590, 14)

In [22]:
# job_str[0:5]

# import manually coded multiple substrings \#1: Mail's data

In [23]:
pathin_multsubstr_marked = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 4 - career/4.3.1 jobs/"
filein_multsubstr_marked = "job_str_1_mergeold_mail.xlsx"

In [24]:
# input mail's parsed & coded data
mail = pd.read_excel(pathin_multsubstr_marked+filein_multsubstr_marked,dtype="str")
mail.shape

(7592, 14)

In [25]:
mail.columns

Index(['CareerStringID', 'CareerString', 'MultipleSubstrings', 'NotJob',
       'CareerDateString_2022', 'CareerDate', 'CareerSubstringID',
       'CareerSubstring', 'OrgString', 'PositionString', 'OrgID', 'OrgName',
       'PositionID', 'PositionName'],
      dtype='object')

In [26]:
# save "not jobs" as a separate df
mail_notjob = mail[mail.NotJob.notnull()]
mail_notjob.shape

(925, 14)

In [27]:
# drop "not jobs" from mail
mail = mail[mail.NotJob.isnull()]
mail.shape

(6667, 14)

In [30]:
# not sure what's going on here. but moving forward.
mail[mail.duplicated(["CareerString","CareerDateString_2022"],keep="first")].shape

(66, 14)

# merge multiple substring field & expand rows

In [31]:
# confirm MultipleSubstring is null in job_str
job_str[job_str.MultipleSubstrings.notnull()].shape

(0, 14)

In [32]:
# select relevant columns: CareerString, MultipleSubstrings
mail = mail[["CareerString","CareerDateString_2022","MultipleSubstrings"]]

In [33]:
mail[0:5]

,CareerString,CareerDateString_2022,MultipleSubstrings
925,"1992.12 정무원 총리 해임, 당 정치국 후보위원으로 강등, 자강도 당 책임비서...",1950,0
926,2003. 3 조선일본군위안부 및 강제연행피해자보상대책위원장,2009.02,0
927,"당 정치국 위원, 당 중앙위원회 부위원장, 당 국제부장 등 해임(추정)",2010.05,0
928,"당 정치국 위원, 당 중앙위원회 부위원장, 당 선전 선동부장 해임(추정)",2018.05,0
929,"당 정치국 후보위원, 당 중앙군사위원회 위원 소환 및 인민무력상 해임(추정)",1947,0


In [35]:
job_str[0:5]

,CareerStringID,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,MultipleSubstrings,CareerSubstringID,CareerSubstring,OrgString,PositionString,OrgID,OrgName,PositionID,PositionName
0,1,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,근로단체부장 해임 * 후임: 리일환,2014,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,노동성 상(유임),1999.02,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,당 정치국 후보위원,2020.01,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,당 중앙위원회 정치국 위원 (*당 중앙위원회 제7기 제4차 전원회의에서 보선),2019.04,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
job_str[job_str.CareerString.isin(mail.CareerString)].shape

(6667, 14)

In [37]:
# bijective ~ CareerStrings in job_str & mail
job_str[~job_str.CareerString.isin(mail.CareerString)].shape

(923, 14)

In [38]:
mail[~mail.CareerString.isin(job_str.CareerString)].shape

(0, 3)

In [39]:
# merge job_str & mail on CareerString
job_str_merge = job_str.merge(mail,on=["CareerString","CareerDateString_2022"],how="left",suffixes=('', '_y'))

# unique on ["CareerString","CareerDateString_2022]

In [40]:
# why is this not 7590
job_str_merge.shape

(7614, 15)

In [41]:
sum(job_str_merge.duplicated(["CareerString","CareerDateString_2022"],keep="first"))

24

In [42]:
# check for duplicates in job_str and mail
sum(mail.duplicated(["CareerString","CareerDateString_2022"],keep="first"))

66

In [43]:
# probably some substrings were coded in the past
sum(job_str.duplicated(["CareerString","CareerDateString_2022"],keep="first"))

0

# expand rows with MultipleSubStrings > 1

In [44]:
sum(job_str_merge.MultipleSubstrings.notnull())

0

In [45]:
job_str_merge["MultipleSubstrings"] = job_str_merge["MultipleSubstrings_y"]

In [46]:
# keep original columns
job_str = job_str_merge.loc[:,job_str_merge.columns.isin(job_str.columns)]
job_str.shape

(7614, 14)

In [47]:
job_str[0:5]

,CareerStringID,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,MultipleSubstrings,CareerSubstringID,CareerSubstring,OrgString,PositionString,OrgID,OrgName,PositionID,PositionName
0,1,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,근로단체부장 해임 * 후임: 리일환,2014,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,노동성 상(유임),1999.02,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,당 정치국 후보위원,2020.01,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,당 중앙위원회 정치국 위원 (*당 중앙위원회 제7기 제4차 전원회의에서 보선),2019.04,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# export single career strings

In [148]:
# export single career strings. (MultipleString is NaN)
job_str_single = job_str[job_str.MultipleSubstrings.isnull()]
job_str_single.shape

(7284, 14)

In [155]:
job_str_single.to_excel(path_job_str+fileout_job_str+"_single.xlsx",index=False,encoding="utf-8")

# export multiple career strings

In [149]:
# export multiple career strings (MultipleString is not null)
job_str_multiple = job_str[job_str.MultipleSubstrings.notnull()]
job_str_multiple.shape

(330, 14)

In [150]:
# MultipleString is > 1
job_str_multiple[job_str_multiple.MultipleSubstrings.astype(int)>1].shape

(330, 14)

In [151]:
job_str_multiple[job_str_multiple.MultipleSubstrings.astype(int)==1].shape

(0, 14)

In [156]:
job_str_multiple.to_excel(path_job_str+fileout_job_str+"_multiple.xlsx",index=False,encoding="utf-8")

# export "not jobs"

In [157]:
# and exporting "not jobs"
mail_notjob.to_excel(path_job_str+fileout_job_str+"_notjob.xlsx",index=False,encoding="utf-8")

# export unique Position Strings

In [159]:
# unique Position Strings
unique_positions = job_str_single["PositionString"].drop_duplicates()
unique_positions.shape

(228,)

In [161]:
# output unique positions in job_str_null (careers which don't have  )
unique_positions.to_excel(path_job_str+fileout_job_str+"_positions.xlsx",index=False,encoding="utf-8")